In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import random
import gudhi as gd
import itertools

import tadasets
from datetime import datetime

import ecc
import ecc_plot
import euler_edges
import cliques_counting as cc

In [ ]:
def compare_methods(points, epsilon, draw_lines=True):
    # ECC
    print("Local Simplex")
    start=datetime.now()
    c_ls = ecc.compute_local_contributions(points, epsilon)
    print("\t time:", datetime.now()-start)
    print("\t contributions list lenght: ", len(c_ls))
     
    e_ls = ecc_plot.euler_characteristic_list_from_all(c_ls)
    print("\t points in the ecc plot:    ", len(e_ls))
    
    ecc_plot.plot_euler_curve(e_ls, draw_lines, "Local Simplex")
    
#     # cliques counting
#     print("\n Cliques counting")
#     start=datetime.now()
#     c = cc.compute_all_contributions_nx(points, epsilon)
#     print("\t time:", datetime.now()-start)
#     print("\t contributions list lenght: ", len(c))
    
#     e_cc = ecc_plot.euler_characteristic_list_from_all(c)
#     print("\t points in the ecc plot:    ", len(e_cc))
    
#     ecc_plot.plot_euler_curve(e_cc, draw_lines, "Cliques counting")
    
    #assert((e == e_cc).all())
    
    
    # edges
    print("\n Edges")
    start=datetime.now()
    c = euler_edges.compute_local_contributions_gd(points, epsilon)
    print("\t time:", datetime.now()-start)
    print("\t contributions list lenght: ", len(c))
    
    e_ee = euler_edges.euler_characteristic_list_from_edges(c)
    print("\t points in the ecc plot:    ", len(e_ee))
    ecc_plot.plot_euler_curve(e_ee, draw_lines, "Edges")    
    
    # full simplex
    print("\n Full simplex")
    start=datetime.now()
    rips_complex = gd.RipsComplex(points=points, max_edge_length=epsilon)
    simplex_tree = rips_complex.create_simplex_tree(max_dimension=len(points))
    
    c_full = simplex_tree.get_filtration()
    
    print("\t time:", datetime.now()-start)
    print("\t contributions list lenght: ", len(c_full))
    
    e_full = ecc_plot.euler_characteristic_list_from_all(c_full)
    print("\t points in the ecc plot:    ", len(e_full))
    
    ecc_plot.plot_euler_curve(e_full, draw_lines, "Full Simplex")

# TEST

## Triangle

In [ ]:
point_cloud = np.array([[0, 0], [4,0], [0,3] ])

In [ ]:
plt.scatter(point_cloud[: , 0], point_cloud[: , 1])

In [ ]:
compare_methods(point_cloud, 10)

## Rhomboid

In [ ]:
point_cloud = np.array([[4,0], [0,3.5], [-2, 0], [-0.5, -3]])

In [ ]:
plt.axis('equal')
labels = ["A", "B", "C", "D"]
for i in range(len(point_cloud)):
    plt.scatter(point_cloud[i][0], point_cloud[i][1], color = "red")
    plt.text(point_cloud[i][0], point_cloud[i][1], labels[i], fontsize =15)


In [ ]:
compare_methods(point_cloud, 10)

## Tetrahedron

In [ ]:
point_cloud = np.array([[0,0,0], [0,1,0], [2,0,0], [0,0,3]])

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter([x[0] for x in point_cloud], [x[1] for x in point_cloud], [x[2] for x in point_cloud])

In [ ]:
compare_methods(point_cloud, 10)

## Points from the example in the slides



In [ ]:
point_cloud = np.array([[0,0], [2,-0.2], [4,0.2], [0,-1.5], [2,-3], [4,-1], [3.8,-2.5]] )

In [ ]:
plt.figure(figsize=(10,5))
plt.axis('equal')
for i in range(len(point_cloud)):
    plt.scatter(point_cloud[i][0], point_cloud[i][1], color = "blue")
    plt.text(point_cloud[i][0], point_cloud[i][1], i)

In [ ]:
compare_methods(point_cloud, 10)

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

# Circle

In [ ]:
point_cloud = []

NUMBER_OF_POINTS = 100

random.seed(42)
for i in range(NUMBER_OF_POINTS):
    angle = random.uniform(0,2*np.pi)
    point_cloud.append([np.cos(angle), np.sin(angle)])
    
point_cloud = np.array(point_cloud)

In [ ]:
plt.axis('scaled')
plt.xlim((-1.2,1.2))
plt.ylim((-1.2,1.2))
plt.scatter(point_cloud[:,0], point_cloud[:,1])

In [ ]:
compare_methods(point_cloud, .1, False)

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

## Double circle

In [ ]:
point_cloud = []

NUMBER_OF_POINTS = 25

random.seed(42)
for i in range(NUMBER_OF_POINTS):
    angle = random.uniform(0,2*np.pi)
    point_cloud.append([np.cos(angle), np.sin(angle)])
    
    angle = random.uniform(0,2*np.pi)
    point_cloud.append([5*np.cos(angle), 5*np.sin(angle)])
    
    
point_cloud = np.array(point_cloud)

In [ ]:
plt.scatter(point_cloud[:, 0], point_cloud[:, 1])

In [ ]:
compare_methods(point_cloud, 2, draw_lines=False)
#rounding error

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

# infinity

In [ ]:
point_cloud = tadasets.infty_sign(n=50, noise=None)

In [ ]:
#plt.axis('scaled')
plt.scatter(point_cloud[:, 0], point_cloud[:, 1])

In [ ]:
compare_methods(point_cloud, 1, draw_lines=False)

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

# n shpere

In [ ]:
point_cloud = tadasets.dsphere(n=100, d=3, r=1, ambient=5, noise = 0)

In [ ]:
compare_methods(point_cloud, 1, draw_lines=False)

In [ ]:
# rounding error

## Hypercube

In [ ]:
# random points in the unit cube

N_POINTS = 1000
DIMENSION = 3

np.random.seed(42)
point_cloud = np.random.rand(N_POINTS,DIMENSION)

In [ ]:
compare_methods(point_cloud, 0.01, draw_lines=False)